# Automate Excel with xlwings

## How does it work?!

![](img/xlwings_frompython_architecture.png)

In [ ]:
import xlwings as xw
import numpy as np
import pandas as pd
import datetime as dt
import time
import sys

In [ ]:
xw.__version__

# The Basics

## xw.view(): Excel as viewer for tabular data

In [ ]:
data = np.random.rand(100, 100)
data

In [ ]:
# Opens a new book
xw.view(data)

In [ ]:
# Reuse an existing sheet (sheets gets cleared with every call)
xw.view(np.random.rand(5, 5), xw.sheets.active)

In [ ]:
xw.view(np.random.rand(3, 3), xw.sheets.active)

## Connect to a Book

In [ ]:
# Fire up a new book in the active Excel instance
wb1 = xw.Book()

In [ ]:
# Connects to an unsaved book (looks in all Excel instances)
wb1 = xw.Book('Book2')

In [ ]:
# Connects to a book by file name or full path and opens it if it is not open yet
# Windows: Use raw strings for path: r'C:\path\to\file.xlsx'
# Again: looks in all Excel instances

wb1 = xw.Book('C:\\Users\\MyWorkbook.xlsx')
# opens the referenced workbook and then, proceeds to read the data in python

## Sheet object

In [ ]:
sheet = wb1.sheets[0]
sheet
# only references an existing sheet in workbook
# does not create a new sheet

## The Range object

In [ ]:
# Write value
sheet.range('A1').value = 'Hello xlwings!'

In [ ]:
# Read value
sheet.range('A1').value

In [ ]:
# Write the same value to multiple cells
sheet.range('A3:B4').value = 123

In [ ]:
# Excel's numerical format is float!
sheet.range('A3').value

In [ ]:
# Datetime
sheet.range('A6').value = dt.datetime(2014, 12, 9, 12, 3, 25)
sheet.range('A6').value

In [ ]:
# Index notation (1-based like Excel!)
# input is tuple type
sheet.range((1,1)).value

In [ ]:
# Formula
sheet.range('B1').formula = '=SUM(A3:B4)'

In [ ]:
# Named ranges
sheet.range('B1').name = 'test'
sheet.range('test').formula

<div class="alert alert-info">

**Named ranges**: Named ranges give you a solid way of reading in values (e.g. parameters) that survive a reorganization of the sheet. The same is true for defining a target cell to write out values.

</div>

In [ ]:
test = sheet.range('test').value
test

In [ ]:
sheet.range('test').value = 'Output'

## 2d Ranges

In [ ]:
sheet.range('A3:B4').value

In [ ]:
# Index notation
sheet.range((3,1),(4,2)).value

In [ ]:
# Assign a nested list to the top-left corner
sheet.range('A9').value = [['a string', 1, 2, 3],
                           [dt.datetime(2010, 1, 1), 123.5, None, None]]

In [ ]:
# Range expansion: 'table', 'down', 'right'
# Correspond to Ctrl-Shift-down and/or right
# They return a Range object!
# the input can be a cell reference or a range 'A9:C10'
sheet.range('A9').expand('table')

In [ ]:
# 'table' is default
sheet.range('A9').expand().value

In [ ]:
# Use .clear() to also clear the formatting
sheet.range('A9').expand().clear_contents()
# can also be applied without the use of expand and with a range of cells as inputs like 'A9:C10'
# clear_contents can also be replaced with clear command

## 1d vectors

In [ ]:
# Horizontal...
sheet.range('A12').value = [1, 2, 3, 4]

In [ ]:
# ... and vertical vectors
sheet.range('A13').options(transpose=True).value = [5, 6, 7, 8]
# this is the same as:
# sheet.range('A13').value = [[5], [6], [7], [8]]

In [ ]:
sheet.range('A12').expand('right').value # returns multiple values in the form of a list

In [ ]:
sheet.range('A12').expand('down').value

## ndim

In [ ]:
sheet.range('A12').options(ndim=2, expand='right').value

In [ ]:
sheet.range('A12').options(ndim=2, expand='down').value

## Autofit

In [ ]:
# autofit columns and rows based on single Cell
sheet.range('A3').autofit()

In [ ]:
# autofit columns based on Range
sheet.range('A1:C3').columns.autofit()

In [ ]:
# autofit a whole column
sheet.range('A:A').autofit()

## Background color

In [ ]:
# Assign an RGB tuple
sheet.range('A1').color = (0, 255, 0)
sheet.range('A1').color

# Set a named range

In [ ]:
sht.range('A1').expand().name ='Table2' # sets the table anchored at A1 to Table2 in the assigned sht variable

# Range indexing/slicing

In [ ]:
rng = sheet.range('A1:D5')
rng[0, 0]

In [ ]:
rng[1]

In [ ]:
rng[:, 3:]

In [ ]:
rng[1:3, 1:3]

>Remember the above range command returns a Range object and not the values in the range; need to use rng.value method to return the range values for usage

In [ ]:
xw.books.active.close()
# closes all active excel workbooks

xw.Book('book3').close()
# closes the open workbook book3 while other workbook remains open

# Full qualification

In [ ]:
# Get all availabe PIDs (Process Ids)
xw.apps.keys()

In [ ]:
# This allows us to specificy a specific Excel instance
pid = xw.apps.keys()[0] # or you could use xw.apps.active.pid

In [ ]:
# We start at the app (=Excel instance) and walk our way down to the range
xw.apps[pid].books[0].sheets[0].range('A1')

In [ ]:
# Square brackets behave like in Python, whereas round brackets behave like in Excel:
xw.apps(pid).books(1).sheets(1).range('A1')

In [ ]:
# Instead of indices we can also use names:
xw.apps[pid].books['Book1'].sheets['Sheet1'].range('A1')
xw.apps(pid).books('Book1').sheets('Sheet1').range('A1')

## Work with multiple apps

In [ ]:
app1 = xw.apps[pid]
app2 = xw.App()

In [ ]:
# Open the same workbook twice in different Excel instances
app1.books.open('timeseries.xlsx')
app2.books.open('timeseries.xlsx')

In [ ]:
# xw.Book('timeseries.xlsx')  # this will throw an error

In [ ]:
# The following syntax is *required* if the same file is open in >1 instances (full qualification)
print(app1.books['timeseries.xlsx'])
print(app2.books['timeseries.xlsx'])
print(app1.books['timeseries.xlsx'].app)
print(app2.books['timeseries.xlsx'].app)

# Active Objects

In [ ]:
# Active app
xw.apps.active

In [ ]:
# active book in active app
xw.books.active

In [ ]:
# active sheet in active book in active app
xw.sheets.active

In [ ]:
# This is a special shortcut for interactive use only:
# It takes the active sheet from the active book
xw.Range('A1').value

In [ ]:
app2.kill()

<div class="alert alert-info">

**Note**: Active objects are meant to be used in interactive use. Don't use them in scripts as this would not be reliable. This is especially true for `xw.Range`. For scripts, always go through a sheet object: `sheet.range(...)`.


</div>

# Sheets

In [ ]:
xw.sheets[0].name # refers to active open workbook
"""
For multiple open workbooks, the code would be \\
    xw.Book[<name>].sheets[0].name
"""

In [ ]:
xw.sheets.count  # or: len(xw.sheets)

In [ ]:
xw.sheets.add(name='New', after='Sheet1')
"""
For multiple open workbooks, the code would be \\
    xw.Book[<name>].sheets.add(...\\)
"""

### Range shortcuts: sheet[...]

In [ ]:
sheet = xw.sheets[0]
sheet['A1']  # same as: sheet.range('A1')

In [ ]:
sheet['A1:B5']

In [ ]:
sheet[0, 1]

In [ ]:
sheet[:10, :10]

# Excel Charts

In [ ]:
wb = xw.Book()
sheet = wb.sheets[0]
sheet.range('A1').value = [['one', 'two'],
                           [1.1, 2.2],
                           [3.3, None]]

In [ ]:
chart = sheet.charts.add()
chart.set_source_data(sheet.range('A1').expand())
chart.chart_type = 'line'
chart.top = sheet.range('A5').top

In [ ]:
chart.chart_type = 'area'

In [ ]:
# available chart types (we're only printing the first 10 here)
xw.constants.chart_types[:10]

In [ ]:
wb.close()

# Matplotlib

In [ ]:
%matplotlib inline
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import matplotlib

# Swap rate example
years = [1, 2, 3, 4, 5, 7, 10]
swap_rate = [0.0079, 0.0094, 0.0107, 0.0119,
             0.013, 0.0151, 0.0174]
years_new = np.linspace(1, 10, num=10)
interpolate = interp1d(years, swap_rate, kind='quadratic')

fig = plt.figure(figsize=(6, 4))
swaprate_plot = plt.plot(years, swap_rate, 'o',
                         years_new, interpolate(years_new), '-')

In [ ]:
wb = xw.Book()
sheet = wb.sheets[0]

In [ ]:
plot = sheet.pictures.add(fig, name='SwapRate', update=True)
# generates a figure object that would be visible using selections pane in the worksheet

In [ ]:
# Fine Tuning
width, height = fig.get_size_inches()
dpi = fig.get_dpi()
sheet.pictures.add(fig, name='SwapRate2', update=True,
                   left=sheet.range('A25').left, top=sheet.range('A25').top,
                   width=width * dpi / 2, height=height * dpi / 2)

In [ ]:
# Alternatively manipulate properties after adding the picture
plot.height = plot.height / 2
plot.width = plot.width / 2

In [ ]:
wb.close()

<div class="alert alert-info">

**Note**: If you set `update=True`, you can resize and position the plot on Excel: subsequent calls to `pictures.add()` with the same name  will update the picture without changing its position or size.


</div>

# Table objects

Excel Table objects aren't officially supported yet, but reading actually works nicely:

<div class="alert alert-info">

**Table Objects**: Named ranges option give you a solid way of reading in values (e.g. parameters) that survive a reorganization of the sheet. The same is applied for table objects with <tablename> being providing as a reference in the command syntax.

</div>

In [ ]:
# The sample book has a table that was created with:
# Insert > Table (incl. Header Row and Total Row)
wb = xw.Book('table_objects.xlsx')
sheet = wb.sheets[0]

In [ ]:
# Get entire table body - no different from named range
sheet.range('Table1').value

In [ ]:
# Get Symbol column data without header and total rows
sheet.range('Table1[Symbol]').value

In [ ]:
# Get 'Last' title column including header and total rows
sheet.range('Table1[[#All], [Last]]').value

# Get all including header and total row
# Possibility to connect table data to KNIME workflow
sheet.range('Table1[#All]').value

# Get all including header and excluding total row
sheet.range('Table1[#All]').value[:-1]

In [ ]:
# note the position of the single quote encompassing the complete table and header details
# Get header row for one column
sheet.range('Table1[[#Headers], [Last]]').value

# Get only header row for all columns
sheet.range('Table1[#Headers]').value

In [ ]:
# Total row for one column
sheet.range('Table1[[#Totals], [Last]]').value

In [ ]:
# Two or more adjacent columns
sheet.range('Table1[[Index]:[Last]]').value

In [ ]:
wb.close()

# Efficiency

<div class="alert alert-info">

**Watch out**: Minimize your cross-application calls to improve efficiency, i.e. if possible, always read and write 2d ranges instead of single cells: `sht.range('A1').value = [[1,2],[3,4]]` instead of `sheet.range('A1').value = 1`, `sheet.range('B1').value = 2` etc. 

</div>

In [ ]:
wb = xw.Book()
sheet = wb.sheets[0]

In [ ]:
# This calls each cell individually - Don't do this!
for i, cell in enumerate(sheet.range('A1:E30')):
    cell.value = i

In [ ]:
sheet.range('A1').expand().clear()

In [ ]:
# Do this instead
import numpy as np
sheet.range('A1').value = np.arange(5 * 30).reshape((30, 5))

In [ ]:
wb.close()


# Workaround for missing features: Manipulate the underlying object

In [ ]:
wb = xw.Book()
sheet = wb.sheets[0]

In [ ]:
# On Windows, the underlying object is a pywin32 COM object
# On Mac, the underlying object is an appscript object

sheet.range('A1').api  # same for the other objects

## E.g. Range.ClearFormats() is not implemented yet

In [ ]:
sheet.range('A10').value = 1
sheet.range('A10').color = (255, 0, 0)

In [ ]:
# Code that makes use of the api property will be platform dependent (!)
if sys.platform.startswith('darwin'):
    # Mac version (appscript syntax)
    sheet.range('A10').api.clear_formats()
elif sys.platform.startswith('win'):
    # Windows version (pywin32 syntax)
    sheet.range('A10').api.ClearFormats()

# Calling VBA macros

Paste the following code into a VBA module:
```
Function MySum(x, y)
    MySum = x + y
End Function
```
Then run you can call it from Python:

In [ ]:
wb = xw.books.active
my_sum = wb.macro('MySum')
my_sum(1, 2)